## Dataset

In [1]:
import time
import random as rd
import csv

In [2]:
def crea_dataset():
    column_names=['server_id', 'application_id', 'start_timestamp']
    server_list = [x for x in range(1,11)]
    application_list = ['app'+ str(x)  for x in range(1,11)]
    start_time_list=[int(time.time())+x for x in range(1,1000)]
    data=[]
    data.append(column_names)
    for i in range(1000):
        raw=[i,rd.choice(server_list),rd.choice(application_list), rd.choice(start_time_list)]
        data.append(raw)
    with open('dataset.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    return data    

In [3]:
crea_dataset();

## Analysis

In [4]:
import numpy as np
import pandas as pd

In [5]:
df= pd.DataFrame.from_csv('dataset.csv')

In [6]:
df.head()

,server_id,application_id,start_timestamp
0,8,app4,1508344483
1,6,app6,1508344610
2,2,app8,1508345115
3,5,app8,1508345186
4,8,app5,1508344661


In [7]:
df.columns

Index(['server_id', 'application_id', 'start_timestamp'], dtype='object')

In [8]:
df=df.sort_values(by=['start_timestamp'], ascending=[True])

In [9]:
df = df.reset_index(drop=True)
df[:5]

,server_id,application_id,start_timestamp
0,10,app6,1508344284
1,10,app1,1508344284
2,8,app2,1508344286
3,10,app1,1508344287
4,5,app7,1508344288


In [10]:
df.groupby(df['server_id']).get_group(2).describe()

,server_id,start_timestamp
count,108.0,1.080000e+02
mean,2.0,1.508345e+09
std,0.0,2.985240e+02
min,2.0,1.508344e+09
25%,2.0,1.508345e+09
50%,2.0,1.508345e+09
75%,2.0,1.508345e+09
max,2.0,1.508345e+09


In [11]:
p=pd.Series()
for i in range(1,11):    
    s=df.groupby(['server_id'])['start_timestamp'].get_group(i).diff().shift(-1)
    p=p.append(s)
p.sort_index()
df['app_time']=p

In [12]:
df['app_time'].describe()

count    990.000000
mean       9.845455
std        9.827293
min        0.000000
25%        3.000000
50%        7.000000
75%       14.000000
max       69.000000
Name: app_time, dtype: float64

In [13]:
df['app_time'].isnull().sum()

10

In [14]:
df_filtered=df[(df.app_time > 45 ) & (df.app_time < 300)]
df_filtered

,server_id,application_id,start_timestamp,app_time
5,8,app1,1508344289,51.0
145,1,app1,1508344421,51.0
153,8,app4,1508344428,50.0
184,9,app5,1508344467,69.0
323,5,app5,1508344604,48.0
356,7,app10,1508344636,49.0
368,5,app1,1508344652,52.0
531,8,app6,1508344811,65.0


In [15]:
df['application_id'].groupby(df['server_id']).describe()

,count,unique,top,freq
server_id,,,,
1,106,10,app9,18
2,108,10,app2,17
3,112,10,app8,17
4,89,10,app8,12
5,99,10,app1,15
6,98,10,app9,13
7,108,10,app2,13
8,84,10,app3,15
9,83,10,app10,10


In [16]:
df.groupby(['server_id', 'application_id'])['app_time'].describe()

count       mean        std  min   25%   50%    75%  \
server_id application_id                                                        
1         app1             10.0  16.200000  14.164117  1.0  8.25  12.0  20.25   
          app10            13.0  10.230769   9.688084  0.0  5.00   8.0  10.00   
          app2              7.0   8.428571   8.638232  0.0  4.00   6.0   9.50   
          app3              7.0   8.571429   6.553807  1.0  4.00   8.0  12.00   
          app4             14.0   7.214286   9.065391  1.0  2.00   3.5   8.50   
          app5             15.0   8.800000   9.645132  0.0  5.50   7.0   8.00   
          app6              8.0   6.125000   5.667892  1.0  2.00   3.5   9.50   
          app7              4.0   6.000000   4.082483  2.0  2.75   6.0   9.25   
          app8              9.0  11.222222  11.333333  1.0  3.00   7.0  14.00   
          app9             18.0   8.611111   9.146770  0.0  2.25   7.0  10.75   
2         app1              9.0   9.111111   8.838049  0.0  3.00   8.0  11.00   
          app10            14.0  11.642857   8.526945  2.0  7.00   8.0  17.25   
          app2             17.0   6.647059   6.670281  0.0  2.00   3.0  10.00   
          app3              6.0   6.166667   7.139094  0.0  1.00   3.5   9.75   
          app4             13.0   9.000000   8.031189  0.0  2.00   8.0  16.00   
          app5              9.0  11.333333   7.549834  1.0  4.00  11.0  17.00   
          app6              8.0   7.250000   6.158618  1.0  2.75   5.5  10.00   
          app7             10.0  12.800000   8.403703  5.0  6.25   9.0  18.25   
          app8             10.0   7.600000   9.203864  1.0  2.00   3.5  10.00   
          app9             11.0   7.454545   6.547727  1.0  1.50   7.0  12.50   
3         app1              9.0   9.222222   5.890199  1.0  4.00  10.0  13.00   
          app10            12.0   7.000000   6.252272  0.0  1.75   6.0  11.25   
          app2              8.0   8.625000  11.648881  1.0  1.75   4.5   9.50   
          app3             11.0   6.090909   4.846742  1.0  2.00   4.0   9.50   
          app4             14.0   8.500000   9.061754  1.0  2.00   4.5  12.50   
          app5              6.0   8.833333  11.788412  0.0  2.00   6.0   7.75   
          app6             11.0   5.090909   4.369314  1.0  1.50   4.0   6.00   
          app7             12.0  10.416667   5.212892  2.0  8.00  11.0  12.00   
          app8             17.0   9.529412   6.965250  0.0  3.00   9.0  15.00   
          app9             11.0  13.000000  11.713240  0.0  4.50  13.0  18.00   
...                         ...        ...        ...  ...   ...   ...    ...   
8         app1              8.0  13.000000  16.936858  1.0  2.75   7.0  13.25   
          app10             4.0   9.000000   7.615773  1.0  4.75   8.0  12.25   
          app2              7.0   7.000000   7.280110  0.0  2.00   3.0  12.50   
          app3             15.0  12.866667  11.025080  1.0  4.50   9.0  19.00   
          app4              7.0  12.571429  17.154758  2.0  2.00   8.0  12.00   
          app5              5.0   9.200000   9.731393  0.0  2.00   5.0  17.00   
          app6              8.0  17.375000  20.444612  1.0  7.00  12.0  15.75   
          app7              5.0   1.800000   2.049390  0.0  0.00   2.0   2.00   
          app8             14.0  15.500000  11.680029  0.0  5.50  14.5  23.25   
          app9             10.0  11.300000  11.804425  0.0  3.25  11.0  13.50   
9         app1              8.0  14.250000   8.746428  6.0  7.50  14.5  15.50   
          app10            10.0   9.100000   6.471304  1.0  4.75   8.5  12.00   
          app2              9.0  10.111111  13.923402  0.0  1.00   5.0  15.00   
          app3              9.0  11.111111  11.826289  2.0  3.00   7.0  11.00   
          app4              8.0   4.500000   2.976095  0.0  2.00   5.0   6.25   
          app5              9.0  23.666667  20.844664  1.0  6.00  25.0  29.00   
          app6              7.0  12.571429   9.253056  3.